In [1]:
from __future__ import division
from mnist import MNIST
from sklearn import tree
import numpy as np
from sklearn.metrics import classification_report,log_loss,accuracy_score,roc_auc_score
from time import time
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
import re
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn import tree

In [2]:
# Loading the data
mndata = MNIST('./')
images_train, labels_train = mndata.load_training()
# images_train = np.array(images_train)
labels_train = np.array(labels_train)

images_test, labels_test = mndata.load_testing()
# images_test = np.array(images_test)
labels_test = np.array(labels_test)



In [10]:
def classifaction_report_to_dataframe(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-5]:
        row = {}
        row_data = line.split('      ')
        if len(row_data)>4:
            row['class'] = row_data[1]
            row['precision'] = float(row_data[2])
            row['recall'] = float(row_data[3])
            row['f1_score'] = float(row_data[4])
            row['support'] = float(row_data[5])
            report_data.append(row)
    for line in lines[-4:-1]:
        row = {}
        p = re.compile("[a-z]+ [a-z]+")
        m = p.search(line)  
        row['class'] = m.group(0)
        p = re.compile("\d+.?\d+")
        row_data = p.findall(line)
        row['precision'] = float(row_data[0])
        row['recall'] = float(row_data[1])
        row['f1_score'] = float(row_data[2])
        row['support'] = float(row_data[3])
        report_data.append(row)
        
        
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe


In [3]:
dimen = [1,2,5,10,15,20,35,30,40,50,60,75,80,100,150,200,250,300,325,350,400,450,500,550,600,700,784]
max_depth_nodes = [1,2,5,10,15,20,25,30,35,40,None]


In [12]:
'''
dimen = [1,2]
max_depth_nodes = [1,2]
'''

'\ndimen = [1,2]\nmax_depth_nodes = [1,2]\n'

In [5]:
excel_file_name = 'DecisionTree_PPCA_' + str(datetime.now().strftime('%Y%m%d%H%M%S')) + '.xlsx'
excel_writer = pd.ExcelWriter(excel_file_name, engine='openpyxl')
df = pd.DataFrame(columns=['Test'])
df.to_excel(excel_writer, sheet_name='demo_sheet')

excel_writer.save()

/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


In [6]:
df = pd.DataFrame(columns=['Max depth nodes','No of dimension', 'Accuracy','Normalized Accuracy', 
                 'Time_taken_train','Time_taken_test',
                 'Negative Log loss','Macro_auc','Micro_auc'])

In [11]:
for d in dimen:
        t1_img = np.fromfile('train_img' + str(d) + '.dat', dtype='float64')
        t1_tst = np.fromfile('test_img' + str(d) + '.dat', dtype='float64')
        images_train = np.reshape(t1_img,(-1,d))
        images_test = np.reshape(t1_tst,(-1,d))
        
        for n in max_depth_nodes:
            clf = tree.DecisionTreeClassifier(max_depth=d)

            start = time()
            clf.fit(images_train, labels_train)
            train_time = time() - start

            start = time()
            predictions = clf.predict(images_test)
            test_time = time() - start
            
            accuracy = accuracy_score(predictions,labels_test,normalize=False)
            normalized_accuracy = accuracy_score(predictions,labels_test,normalize=True)
            log_loss_prediction = clf.predict_proba(images_test)
            log_loss_val = log_loss(labels_test,log_loss_prediction) 
            classfication_repo = classification_report(labels_test, predictions)



            labels_train_one_hot=label_binarize(labels_test,classes=[0,1,2,3,4,5,6,7,8,9])
            predictions_one_hot=label_binarize(predictions,classes=[0,1,2,3,4,5,6,7,8,9])
            micro_auc=roc_auc_score(labels_train_one_hot,predictions_one_hot,average="micro")
            macro_auc=roc_auc_score(labels_train_one_hot,predictions_one_hot,average="macro")

            cm = confusion_matrix(labels_test,predictions)
            cm_df = pd.DataFrame(data=cm)
            
            
            df_data = [n,d,accuracy,normalized_accuracy,train_time,test_time,log_loss_val,macro_auc,micro_auc]
            df.loc[len(df)] = df_data
    
            book = load_workbook(excel_file_name)
            with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
                excel_writer.book = book
                df_classification_report = classifaction_report_to_dataframe(classfication_repo)
                df_classification_report.to_excel(excel_writer, sheet_name='dt_ppca_cr' + '_'+ str(n) + '_'+ str(d))

                cm_df.to_excel(excel_writer, sheet_name='dt_ppca_cr' + '_'+ str(n)+'_'+ str(d),
                               startrow=len(df_classification_report)+5)

                excel_writer.save()    
    
book = load_workbook(excel_file_name)
with pd.ExcelWriter(excel_file_name, engine='openpyxl') as excel_writer:
    excel_writer.book = book
    df.to_excel(excel_writer, sheet_name='dt_ppca')
    excel_writer.save()   

/users/mtech/sridivyay/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])
/users/mtech/sridivyay/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/lib/python3/dist-packages/pandas/io/excel.py:687: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])
/users/mtech/sridivyay/.local/lib/python